In [4]:
# test selenium
import selenium
print(selenium.__version__)

3.13.0


In [160]:
# import
from selenium import webdriver
import numpy as np
import cv2
import dlib
import os
import math

# for selenium
chrome_driver_path = 'C:\chromedriver\chromedriver.exe'
common_url = 'https://google.com'
common_rawdata_path = "./data/webdata/crawling_data/raw_data/"
common_facedata_path = "./data/webdata/crawling_data/faces/"

# for dlib
ALL = list(range(0, 68))
RIGHT_EYEBROW = list(range(17, 22))
LEFT_EYEBROW = list(range(22, 27))
NOSE = list(range(27, 36))
RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
MOUTH_OUTLINE = list(range(61, 68))
JAWLINE = list(range(0, 17))
INDEX = ALL

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
outer_border = 50
inner_border = 100


In [161]:
def check_face(faces, path) :
    if len(faces) == 0 :
        print("zero face detected in ", path)
        return -1
    else : print(str(len(faces)) + " faces are detected.")
    return 0

In [162]:
def crop_ppl(f) :
    w = f.right() - f.left()
    h = f.bottom() - f.top()
    if w > h : h = w
    else : w = h
    x1 = f.left() - w
    y1 = f.top() - h
    x2 = f.left() + 2*w
    y2 = f.top() + 2*h
    
    crop_img = img[y1:y2, x1:x2]
    return crop_img

In [163]:
def get_landmarks(face, img) :
    landmarks = predictor(img, face)
    landmark_list = []
    
    for p in landmarks.parts():
        landmark_list.append([p.x, p.y])
    landmark_list = np.array(landmark_list)

    face_mid = []
    for i, pt in enumerate(landmark_list[INDEX]):
        pt_pos = (pt[0], pt[1])
        if i == 33 : nose_end = pt_pos
        elif i == 27 : nose_start = pt_pos
        elif i == 16 : face_right = pt_pos
        elif i == 9 : face_mid.append(pt_pos)
        elif i == 8 : face_mid.append(pt_pos)
        elif i == 7 : face_mid.append(pt_pos)
        elif i == 0 : face_left = pt_pos
                
    return (face_left, face_right, nose_start, nose_end, face_mid)

In [164]:
def draw_landmarks(faces, img) :
    print(str(len(faces)) + " faces are here!!")
    for face in faces : 
        landmarks = predictor(img, face)
        landmark_list = []
        
        for p in landmarks.parts():
            landmark_list.append([p.x, p.y])
        landmark_list = np.array(landmark_list)

        for i, pt in enumerate(landmark_list[INDEX]):
            pt_pos = (pt[0], pt[1])
            if i==0 :
                point_f = pt_pos
                cv2.circle(img, pt_pos, 2, (255, 0, 0), -1)
            elif i==27 : 
                point_s = pt_pos
                cv2.circle(img, pt_pos, 2, (0, 0, 255), -1)
            elif i==33: 
                point_e = pt_pos
                cv2.circle(img, pt_pos, 2, (0, 0, 255), -1)
            else :
                cv2.circle(img, pt_pos, 2, (0, 255, 0), -1)       
                
    #cv2.imwrite(landmark_path, img_landmark)
    return (point_f, point_s, point_e)

In [165]:
def face_alignment(origin_w, origin_h, nose_start, nose_end, img) :
    # vertical alignment
    isRotated = False
    angle = math.degrees(math.atan((nose_start[0] - nose_end[0])/([nose_end[1] - nose_start[1]])))
    
    if abs(angle)>=3 :
        matrix = cv2.getRotationMatrix2D((nose_start[0], nose_start[1]), angle, 1)
        rotated_img = cv2.warpAffine(img, matrix, (origin_w, origin_h))
        isRotated = True
        return (isRotated, rotated_img)
        
    return (isRotated, img)

In [166]:
def crop_tight(face_right, face_left, nose_start, face_mid, img) :
    mid_jaw_y = max(face_mid[0][1], face_mid[1][1], face_mid[2][1])
    stand_h = mid_jaw_y - nose_start[1]
    stand_w = 2.5 * stand_h / 2
    start_x = nose_start[0] - stand_w
    end_x = nose_start[0] + stand_w
    start_y = nose_start[1] - int(1.2 * stand_h)
    end_y = nose_start[1] + int(1.2 * stand_h)
    
    print("stand_w :", stand_w, " stand_h : ", stand_h, " start_x : ", start_x, " end_x : ", end_x, " start_y : ", start_y, " end_y : ", end_y)
    try:
        ## crop
        cropped_img = img[int(start_y):int(end_y), int(start_x):int(end_x)]
        ## resize
        result_img = cv2.resize(cropped_img, dsize=(256, 256), interpolation = cv2.INTER_AREA)
        return result_img
    except Exception as e :
        print(str(e))
        return img

In [167]:
img_index = 1
origin_img = cv2.imread("./test3.jpeg")

# 사이즈 초기화
h, w, c = origin_img.shape
if (h < 256) & (w < 256) :
    _fx = 256*w/h
    _fy = 256/h
    origin_img = cv2.resize(origin_img, dsize=(0, 0), fx = _fx, fy = _fy, interpolation = cv2.INTER_LINEAR)

# 테두리 추가
img = cv2.copyMakeBorder(origin_img, outer_border, outer_border, outer_border, outer_border, cv2.BORDER_REPLICATE)
cv2.imwrite("./test_bordered.png", img)

# 얼굴 인식
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = detector(img_gray, 1)
if check_face(faces, "test") < 0 : print("end")
else :
    for face_index, face in enumerate(faces) :
        # landmark 값 가져오기
        origin_w, origin_h, origin_c = img.shape
        face_left, face_right, nose_start, nose_end, face_mid = get_landmarks(face, img)
        # rotate 이미지 생성
        isRotated, rotated_img = face_alignment(origin_w, origin_h, nose_start, nose_end, crop_img)
        cv2. imwrite("./test_rotated_"+str(face_index).zfill(2)+".png", rotated_img)
        
        if isRotated :
            img_gray_2 = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2GRAY)
            faces_2 = detector(img_gray_2, 1)
            face_left2, face_right2, nose_start2, nose_end2, face_mid2 = get_landmarks(faces_2[0], rotated_img)
            result = crop_tight(face_right2, face_left2, nose_start2, face_mid2, rotated_img)
        else :
            result = crop_tight(face_right, face_left, nose_start, face_mid, img)

        path = "./test" + "_" + str(face_index).zfill(2) + ".png"
        cv2.imwrite(path, result)
        print(path)
    
        print("end")

1 faces are detected.
stand_w : 106.25  stand_h :  85  start_x :  27.75  end_x :  240.25  start_y :  48  end_y :  252
./test_00.png
end


In [158]:
search_list = ['soccer injury', 'olympic injury', 'women soccer injury', 'baseball injury']

for search_term in search_list :
    rawdata_path = common_rawdata_path + search_term + "/"
    facedata_path = common_facedata_path + search_term + "/"
    url = common_url + "/search?q=" + search_term + "&tbm=isch"
    driver = webdriver.Chrome(chrome_driver_path)
    driver.get(url)
    
    # scroll down for more imgs
    for i in range(10) : 
        browser.execute_script("window.scrollBy(0, 1000)")
    
    for img_index, img in enumerate(brower.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")) :
        index = str(img_index).zfill(5)
        rawdata_file = rawdata_path + "_" + index + ".png"
        img.screenshot(rawdata_file)
        
        origin_img = cv2.imread(rawdata_file)
        img = cv2.copyMakeBorder(origin_img, outer_border, outer_border, outer_border, outer_border, cv2.BORDER_REPLICATE)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detector(img_gray, 1)
        if(check_face(faces, rawdata_file) < 0) : continue
        
        for face_index, face in enumerate(faces) :
            crop_img = crop_ppl(face)
            face_left, face_right, nose_start, nose_end, face_mid = get_landmarks(face, crop_img)
            draw_landmarks(crop_img, face)
            isRotated, rotated_img = face_alignment(origin_w, origin_h, nose_start, nose_end, crop_img)
            result = crop_tight(isRotated, face_right, face_left, nose_start, face_mid, rotated_img)
            
            facedata_file = facedata_path + "_" + index + "_" + str(face_index).zfill(2) + ".png"
            cv2.imwrite(facedata_file, result)
    
    driver.close()

NameError: name 'browser' is not defined